<a href="https://colab.research.google.com/github/Pavadareni/AI_Training/blob/main/RNN%20Quantization/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#normal rnn model

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError


def get_dataset(num_samples):
    x = np.random.rand(num_samples, 50, 1)
    y = np.sum(x, axis=1)
    return x, y

x_train, y_train = get_dataset(1000)
x_test, y_test = get_dataset(200)

epochs = 20
batch_size = 32
learning_rate = 0.001

seq_length = x_train.shape[1]
input_dim = x_train.shape[2]

model = Sequential([
    SimpleRNN(64, input_shape=(seq_length, input_dim), activation='relu'),
    Dense(1)
])

model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss=MeanSquaredError(),
    metrics=[MeanAbsoluteError()]
)

history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)
model.save('rnn_model.h5')

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 595.1750 - mean_absolute_error: 24.0348 - val_loss: 292.2782 - val_mean_absolute_error: 17.0058
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 157.1952 - mean_absolute_error: 11.1721 - val_loss: 7.5851 - val_mean_absolute_error: 2.6083
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.9013 - mean_absolute_error: 1.6774 - val_loss: 1.4150 - val_mean_absolute_error: 0.9769
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.8691 - mean_absolute_error: 0.7457 - val_loss: 0.7364 - val_mean_absolute_error: 0.6934
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.5256 - mean_absolute_error: 0.5822 - val_loss: 0.6408 - val_mean_absolute_error: 0.6556
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.5258 - mean_absolute_error: 0.5917 - val_loss: 0.6577 - val_mean_absolute_error: 0.6572
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.5189 - mean_absolute_error: 0.5791 - val_loss: 

In [6]:
#Quantization
import tensorflow as tf
model = tf.keras.models.load_model('rnn_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()
with open("quantized_rnn_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model converted and saved as quantized_rnn_model.tflite")


Saved artifact at '/tmp/tmpquqgfztt'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 50, 1), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136934375366624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136934395793248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136934375374016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136934375383360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136934375381424: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model converted and saved as quantized_rnn_model.tflite
